<a href="https://colab.research.google.com/github/rummannaqvi/Distilbert-Text-Classification-Fine-Tuning/blob/main/Multi_class_text_classifica_fine_tuning_distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from transformers import (
    DistilBertTokenizerFast,
    TFDistilBertForSequenceClassification,
    pipeline
)

import nltk
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
nltk.download("stopwords")
from nltk.corpus import stopwords
stopw = stopwords.words("english")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
root_path = "bbc-text.csv"
df = pd.read_csv(root_path)


In [5]:
df["encoded_text"] = df["category"].astype("category").cat.codes
num_labels = df["encoded_text"].nunique()


In [6]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"].tolist(), df["encoded_text"].tolist(),
    test_size=0.2, random_state=42
)


In [7]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [8]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)


In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    tf.convert_to_tensor(train_labels)
)).shuffle(1000).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    tf.convert_to_tensor(val_labels)
)).batch(16)


In [11]:
model = TFDistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels,
    use_safetensors=False
)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_39', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)

Epoch 1/3
69/69 [==============================] - 124s 1s/step - loss: 0.5504 - accuracy: 0.8886 - val_loss: 0.1143 - val_accuracy: 0.9672
Epoch 2/3
69/69 [==============================] - 67s 971ms/step - loss: 0.0893 - accuracy: 0.9808 - val_loss: 0.1035 - val_accuracy: 0.9708
Epoch 3/3
69/69 [==============================] - 66s 958ms/step - loss: 0.0336 - accuracy: 0.9963 - val_loss: 0.0688 - val_accuracy: 0.9745


In [13]:
save_directory = "./saved_models"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


('./saved_models/tokenizer_config.json',
 './saved_models/special_tokens_map.json',
 './saved_models/vocab.txt',
 './saved_models/added_tokens.json',
 './saved_models/tokenizer.json')

In [14]:
tokenizer_fine_tuned = DistilBertTokenizerFast.from_pretrained(save_directory)
model_fine_tuned = TFDistilBertForSequenceClassification.from_pretrained(save_directory)


Some layers from the model checkpoint at ./saved_models were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./saved_models and are newly initialized: ['dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
test_text = val_texts[0]
inputs = tokenizer_fine_tuned(test_text, return_tensors="tf", truncation=True, padding=True)
outputs = model_fine_tuned(inputs)
prediction_value = tf.argmax(outputs.logits, axis=1).numpy()[0]


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [16]:
print("Predicted label:", prediction_value)


Predicted label: 4


In [17]:
classifier = pipeline("text-classification", model=model_fine_tuned, tokenizer=tokenizer_fine_tuned)
print(classifier(test_text))

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Device set to use 0


[{'label': 'LABEL_4', 'score': 0.9878628253936768}]
